In [114]:
import os


In [115]:
os.chdir(r"D:\ML Projects\KidneyDiseaseDL\KidneyDiseaseEndtoEndDL")


In [116]:
%pwd


'D:\\ML Projects\\KidneyDiseaseDL\\KidneyDiseaseEndtoEndDL'

In [117]:
%pip install dagshub

Note: you may need to restart the kernel to use updated packages.


In [118]:
%pip install mlflow

In [119]:
import mlflow


os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/anujayavidmal2002/KidneyDiseaseEndtoEndDL.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="anujayavidmal2002"
os.environ["MLFLOW_TRACKING_PASSWORD"]="deca8a7bc2c45db5f9e58e36d687740f2c9fb087"

mlflow.set_tracking_uri("https://dagshub.com/anujayavidmal2002/KidneyDiseaseEndtoEndDL.mlflow")

# ✅ Test MLflow logging
with mlflow.start_run():
    mlflow.log_param("example_param", 123)
    mlflow.log_metric("example_metric", 0.95)

2025/07/28 14:55:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run luxuriant-goat-751 at: https://dagshub.com/anujayavidmal2002/KidneyDiseaseEndtoEndDL.mlflow/#/experiments/0/runs/c35bd984fff44544b99306aba4a30ccd.
2025/07/28 14:55:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/anujayavidmal2002/KidneyDiseaseEndtoEndDL.mlflow/#/experiments/0.


In [120]:
import tensorflow as tf


In [121]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [122]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [123]:
os.getcwd()

'D:\\ML Projects\\KidneyDiseaseDL\\KidneyDiseaseEndtoEndDL'

In [124]:
import sys

sys.path.append("D:/ML Projects/KidneyDiseaseDL/KidneyDiseaseEndtoEndDL/src")


In [125]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [126]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [127]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [130]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [131]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-07-28 15:00:02,565: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2025-07-28 15:00:02,567: INFO: common: yaml file: params.yaml loaded successfully:]
[2025-07-28 15:00:02,568: INFO: common: created directory at: artifacts:]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 7s 815ms/step - loss: 3.7141 - accuracy: 0.5036
[2025-07-28 15:00:10,336: INFO: common: json file saved at: scores.json:]


2025/07/28 15:00:11 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-07-28 15:00:12,852: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.:]
INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpppv46bhw\model\data\model\assets
[2025-07-28 15:00:13,304: INFO: builder_impl: Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpppv46bhw\model\data\model\assets:]


2025/07/28 15:00:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/28 15:01:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run chill-rat-297 at: https://dagshub.com/anujayavidmal2002/KidneyDiseaseEndtoEndDL.mlflow/#/experiments/0/runs/f3a98d18c07142128046cf2ccd6d0ca2.
2025/07/28 15:01:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/anujayavidmal2002/KidneyDiseaseEndtoEndDL.mlflow/#/experiments/0.
